In [1]:
!pip install prometheus_client opentelemetry-api opentelemetry-sdk

In [2]:

import time
import logging
import uuid
from prometheus_client import Counter, Histogram
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter


In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger("colab-agent-observability")

In [4]:
resource = Resource(attributes={"service.name": "colab-agent-demo"})
provider = TracerProvider(resource=resource)
processor = SimpleSpanProcessor(ConsoleSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
tracer = trace.get_tracer(__name__)

In [5]:
REQUEST_COUNTER = Counter("agent_requests_total", "Total agent requests", ["agent_id", "status"])
REQUEST_DURATION = Histogram("agent_request_duration_seconds", "Duration", ["agent_id"])
TOOL_CALLS = Counter("agent_tool_calls_total", "Tool calls", ["agent_id", "tool", "status"])


In [6]:
class MockLLM:
    def generate(self, prompt: str) -> str:
        if "quantum" in prompt.lower():
            return "ACTION: use_tool:web_search | QUERY: latest quantum computing papers"
        if "hello" in prompt.lower():
            return "RESPONSE: Hello! How are you?"
        return "RESPONSE: You asked: " + prompt


In [7]:
class MockSearchTool:
    def search(self, query: str):
        time.sleep(0.2)
        if "quantum" in query.lower():
            return [
                {"title": "Quantum Paper 1", "url": "https://example.org/q1"},
                {"title": "Quantum Paper 2", "url": "https://example.org/q2"},
            ]
        return []


In [8]:
class SimpleAgent:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = tools

    def run(self, user_id: str, prompt: str, agent_id="demo-agent"):
        request_id = str(uuid.uuid4())
        logs = []
        success = False

        with tracer.start_as_current_span("agent_run", attributes={"agent.id": agent_id, "request.id": request_id}):
            start = time.time()

            # Step 1 — Ask LLM
            llm_out = self.llm.generate(prompt)
            logs.append(f"LLM Output: {llm_out}")
            logger.info(f"LLM Output: {llm_out}")

            # Step 2 — Parse
            if llm_out.startswith("ACTION:"):
                tool_name = "web_search"
                query = llm_out.split("QUERY:")[-1].strip()

                with tracer.start_as_current_span("tool_call", attributes={"tool": tool_name}):
                    try:
                        result = self.tools["web_search"].search(query)
                        TOOL_CALLS.labels(agent_id, tool_name, "success").inc()

                        if result:
                            success = True
                            result_text = f"Found {len(result)} results. First: {result[0]['title']}"
                        else:
                            result_text = "No results found."
                    except Exception as e:
                        TOOL_CALLS.labels(agent_id, tool_name, "error").inc()
                        result_text = str(e)

            elif llm_out.startswith("RESPONSE:"):
                success = True
                result_text = llm_out.replace("RESPONSE:", "").strip()

            else:
                result_text = "Unknown LLM Output"

            # Metrics update
            duration = time.time() - start
            REQUEST_DURATION.labels(agent_id).observe(duration)
            REQUEST_COUNTER.labels(agent_id, "success" if success else "failure").inc()

        return {
            "request_id": request_id,
            "agent_id": agent_id,
            "result": result_text,
            "logs": logs,
            "success": success,
            "metrics_info": f"Duration: {duration:.3f}s"
        }


In [10]:
llm = MockLLM()
tools = {"web_search": MockSearchTool()}
agent = SimpleAgent(llm, tools)

prompt = "Find quantum computing papers"
output = agent.run("user-1", prompt)

print("💡 RESULT:")
print(output)

{
    "name": "tool_call",
    "context": {
        "trace_id": "0x06d4533276b5d2448e6237f135a499fc",
        "span_id": "0x04060601220955b1",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xd1761555b38e185e",
    "start_time": "2025-11-14T12:31:32.776997Z",
    "end_time": "2025-11-14T12:31:32.977224Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "tool": "web_search"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "service.name": "colab-agent-demo"
        },
        "schema_url": ""
    }
}
{
    "name": "agent_run",
    "context": {
        "trace_id": "0x06d4533276b5d2448e6237f135a499fc",
        "span_id": "0xd1761555b38e185e",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-11-14T12:31:32.776864Z",
    "end_time": "2025-11-14T12:31:32.978561Z",
    "status": {
        "status_code": "UNSET"